In [ ]:
import mysql.connector


remote = mysql.connector.connect(
    host = "database-1.c3wcyagu0dve.ap-northeast-2.rds.amazonaws.com",
    port = "3306",
    user = "root",
    password = "Rrnrrn1!",
    database = "amrbase"
)       

<JS 강제 클릭>
driver.execute_script("arguments[0].click();", element)

>브라우저의 자바스크립트 실행 기능을 이용해서,
Selenium이 일반적으로 .click()으로 클릭하지 못하는 상황에
브라우저 내부에서 직접 클릭 이벤트를 발생시키는 방법


In [ ]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

# 결과 저장 리스트
brand_list = []
menu_list = []
price_list = []

# 드라이버 실행 및 창 최대화
driver = webdriver.Chrome(service=Service("../driver/chromedriver"))
driver.maximize_window()
wait = WebDriverWait(driver, 10)

# 카카오 선물하기 접속
driver.get("https://gift.kakao.com/")

# 브랜드 리스트 (검색어 + 결과 필터 겸용)
brands = [
    "스타벅스",
    "이디야커피",
    "빽다방",
    "커피빈",
    "할리스",
    "메가MGC커피",
    "컴포즈커피",
    "더벤티",
    "탐앤탐스"
]

# 메뉴 키워드 리스트
menu_keywords = [
    "아메리카노",
    "카페라떼",
    "카페모카"
]

# 검색 및 정보 추출 함수
def search_and_extract(brand_keyword, menu_keyword):

    # 검색 버튼 클릭
    search_button = wait.until(EC.element_to_be_clickable((By.CLASS_NAME, "link_search")))
    search_button.click()

    # 검색창 입력
    search_input = wait.until(EC.presence_of_element_located((By.ID, "searchInput")))
    search_input.clear()
    search_input.send_keys(brand_keyword + " " + menu_keyword)
    search_input.send_keys(Keys.ENTER)

    # 결과 로딩 대기
    wait.until(EC.presence_of_element_located((By.CLASS_NAME, "btn_cate")))
    wait.until(EC.invisibility_of_element_located((By.CLASS_NAME, "layer_autocomplete")))

    # 가격 필터 열기 버튼 클릭 (JS 강제 클릭) - 그냥 click시 오류 발생
    price_filter_btn = wait.until(EC.element_to_be_clickable((By.CLASS_NAME, "btn_cate")))
    time.sleep(0.5)
    driver.execute_script("arguments[0].click();", price_filter_btn)

    # 최대 가격 입력
    max_price_input = wait.until(EC.presence_of_element_located((By.ID, "maxPrice")))
    max_price_input.clear()
    max_price_input.send_keys("7000")

    # 적용 버튼 클릭
    apply_btn = wait.until(EC.element_to_be_clickable((By.CLASS_NAME, "btn_apply")))
    driver.execute_script("arguments[0].click();", apply_btn)

    # 결과 로딩 대기
    wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, "link_info")))

    total = len(driver.find_elements(By.CLASS_NAME, "link_info"))

    for i in range(total):
        try:
            product_links = driver.find_elements(By.CLASS_NAME, "link_info")
            link = product_links[i]

            # 브랜드명 확인
            brand_elem = link.find_elements(By.CLASS_NAME, "txt_brand")
            if not brand_elem:
                continue
            brand = brand_elem[0].text

            if brand_keyword not in brand:
                continue

            # 메뉴명 확인
            title_elem = link.find_elements(By.CLASS_NAME, "txt_prdname")
            if not title_elem:
                continue
            title = title_elem[0].text

            title_clean = title.replace(" ", "")
            menu_keyword_clean = menu_keyword.replace(" ", "")

            # 제외 조건
            if (
                "+" in title or 
                len(title) > 20 or 
                menu_keyword_clean not in title_clean or
                "디카페인" in title or 
                "2잔" in title
            ):
                continue

            # 가격 확인
            price_elem = link.find_elements(By.CLASS_NAME, "num_price")
            price = price_elem[0].text if price_elem else "가격 없음"

            # 리스트에 저장
            brand_list.append(brand)
            menu_list.append(title)
            price_list.append(price)

        except Exception as e:
            print(f"오류 발생: {e}")
            continue

# 브랜드 x 메뉴 키워드 조합 반복 실행
for brand in brands:
    for menu in menu_keywords:
        try:
            search_and_extract(brand, menu)
        except Exception as e:
            print(f"검색 중 오류 발생: {e}")
        time.sleep(2)

# 종료
driver.quit()

# 데이터프레임 생성
df = pd.DataFrame({
    "브랜드": brand_list,
    "메뉴명": menu_list,
    "가격": price_list
})



In [46]:
# 결과 확인
df

,브랜드,메뉴명,가격
0,스타벅스,아이스 카페 아메리카노 T,"4,700원"
1,스타벅스,카페 아메리카노 T,"4,700원"
2,스타벅스,아이스 카페 라떼 T,"5,200원"
3,스타벅스,카페 라떼 T,"5,200원"
4,스타벅스,아이스 카페 모카 T,"5,700원"
...,...,...,...
65,더벤티,카페모카,"3,500원"
66,탐앤탐스,아메리카노 T (ICE/HOT),"4,600원"
67,탐앤탐스,아메리카노 T (ICE/HOT),"4,600원"
68,탐앤탐스,카페라떼 T (ICE/HOT),"5,200원"


In [48]:
df.to_csv("../data/coffee_price.csv", sep=",")

In [60]:
import mysql.connector

remote = mysql.connector.connect(
    host = "database-1.c3wcyagu0dve.ap-northeast-2.rds.amazonaws.com",
    port = "3306",
    user = "root",
    password = "Rrnrrn1!",
    database = "amrbase"
)

cur = remote.cursor(buffered=True)

In [ ]:
# coffee_price TABLE 생성
cur.execute("CREATE TABLE coffee_price(Brand varchar(16), menu varchar(64), price int)")

In [61]:
# TABLE에 데이터 insert
sql = """INSERT INTO coffee_price VALUES (%s, %s, %s)"""

for i, row in df.iterrows():
    cur.execute(sql, tuple(row))
    print(tuple(row))
    remote.commit()

('스타벅스', '아이스 카페 아메리카노 T', '4,700원')
('스타벅스', '카페 아메리카노 T', '4,700원')
('스타벅스', '아이스 카페 라떼 T', '5,200원')
('스타벅스', '카페 라떼 T', '5,200원')
('스타벅스', '아이스 카페 모카 T', '5,700원')
('스타벅스', '카페 모카 T', '5,700원')
('스타벅스', '아이스 카페 모카 T', '5,700원')
('이디야커피', '아메리카노 ICED (L)', '3,200원')
('이디야커피', '아메리카노 ICED (EX)', '4,200원')
('이디야커피', '아메리카노 HOT (L)', '3,200원')
('이디야커피', '아메리카노 HOT (EX)', '4,200원')
('이디야커피', '카페라떼 ICED (L)', '4,200원')
('이디야커피', '카페라떼 HOT (L)', '4,200원')
('이디야커피', '카페라떼 ICED (EX)', '5,400원')
('이디야커피', '연유카페라떼 ICED (L)', '4,500원')
('이디야커피', '연유카페라떼 ICED (EX)', '5,700원')
('이디야커피', '연유카페라떼 HOT (L)', '4,500원')
('이디야커피', '카페라떼 HOT (EX)', '5,400원')
('이디야커피', '연유카페라떼 HOT (EX)', '5,700원')
('이디야커피', '카페모카 ICED (L)', '4,500원')
('이디야커피', '카페모카 ICED (EX)', '5,700원')
('이디야커피', '카페모카 HOT (L)', '4,500원')
('이디야커피', '카페모카 HOT (EX)', '5,700원')
('빽다방', '아메리카노(ICED)', '2,000원')
('빽다방', '아메리카노(HOT)', '1,700원')
('빽다방', '카페라떼(ICED)', '3,200원')
('빽다방', '카페라떼(HOT)', '3,200원')
('빽다방', '블랙펄카페라떼', '4,600원')
('빽다방',

In [62]:
cur.execute("select * from coffee_price")

result = cur.fetchall()
for row in result:
    print(row)

('스타벅스', '아이스 카페 아메리카노 T', 4)
('스타벅스', '카페 아메리카노 T', 4)
('스타벅스', '아이스 카페 라떼 T', 5)
('스타벅스', '카페 라떼 T', 5)
('스타벅스', '아이스 카페 모카 T', 5)
('스타벅스', '카페 모카 T', 5)
('스타벅스', '아이스 카페 모카 T', 5)
('이디야커피', '아메리카노 ICED (L)', 3)
('이디야커피', '아메리카노 ICED (EX)', 4)
('이디야커피', '아메리카노 HOT (L)', 3)
('이디야커피', '아메리카노 HOT (EX)', 4)
('이디야커피', '카페라떼 ICED (L)', 4)
('이디야커피', '카페라떼 HOT (L)', 4)
('이디야커피', '카페라떼 ICED (EX)', 5)
('이디야커피', '연유카페라떼 ICED (L)', 4)
('이디야커피', '연유카페라떼 ICED (EX)', 5)
('이디야커피', '연유카페라떼 HOT (L)', 4)
('이디야커피', '카페라떼 HOT (EX)', 5)
('이디야커피', '연유카페라떼 HOT (EX)', 5)
('이디야커피', '카페모카 ICED (L)', 4)
('이디야커피', '카페모카 ICED (EX)', 5)
('이디야커피', '카페모카 HOT (L)', 4)
('이디야커피', '카페모카 HOT (EX)', 5)
('빽다방', '아메리카노(ICED)', 2)
('빽다방', '아메리카노(HOT)', 1)
('빽다방', '카페라떼(ICED)', 3)
('빽다방', '카페라떼(HOT)', 3)
('빽다방', '블랙펄카페라떼', 4)
('빽다방', '아이스크림카페라떼', 4)
('빽다방', '카페라떼(ICED)', 3)
('빽다방', '카페라떼(HOT)', 3)
('빽다방', '카페모카(ICED)', 3)
('빽다방', '아이스크림카페모카', 4)
('빽다방', '카페모카(HOT)', 3)
('커피빈', '블랙다이몬 아메리카노', 6)
('커피빈', '아이스 아메리카노 (S)', 5)
('커

In [63]:
df_final = pd.DataFrame(result)
df_final.head()

,0,1,2
0,스타벅스,아이스 카페 아메리카노 T,4
1,스타벅스,카페 아메리카노 T,4
2,스타벅스,아이스 카페 라떼 T,5
3,스타벅스,카페 라떼 T,5
4,스타벅스,아이스 카페 모카 T,5


In [64]:
remote.commit()

In [65]:
remote.close()

In [66]:
import mysql.connector

remote = mysql.connector.connect(
    host = "database-1.c3wcyagu0dve.ap-northeast-2.rds.amazonaws.com",
    port = "3306",
    user = "root",
    password = "Rrnrrn1!",
    database = "amrbase"
)

cur = remote.cursor(buffered=True)

# coffee_price TABLE 생성
cur.execute("CREATE TABLE coffee_price(Brand varchar(16), menu varchar(64), price int)")

# TABLE에 데이터 insert
sql = """INSERT INTO coffee_price VALUES (%s, %s, %s)"""

for i, row in df.iterrows():
    cur.execute(sql, tuple(row))
    remote.commit()

remote.commit()
remote.close()


In [ ]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import mysql.connector

# 결과 저장 리스트
brand_list = []
menu_list = []
price_list = []

# 드라이버 실행 및 창 최대화
driver = webdriver.Chrome(service=Service("../driver/chromedriver"))
driver.maximize_window()
wait = WebDriverWait(driver, 10)

# 카카오 선물하기 접속
driver.get("https://gift.kakao.com/")

# 브랜드 리스트 (검색어 + 결과 필터 겸용)
brands = [
    "스타벅스",
    "이디야커피",
    "빽다방",
    "커피빈",
    "할리스",
    "메가MGC커피",
    "컴포즈커피",
    "더벤티",
    "탐앤탐스"
]

# 메뉴 키워드 리스트
menu_keywords = [
    "아메리카노",
    "카페라떼",
    "카페모카"
]

# 검색 및 정보 추출 함수
def search_and_extract(brand_keyword, menu_keyword):

    # 검색 버튼 클릭
    search_button = wait.until(EC.element_to_be_clickable((By.CLASS_NAME, "link_search")))
    search_button.click()

    # 검색창 입력
    search_input = wait.until(EC.presence_of_element_located((By.ID, "searchInput")))
    search_input.clear()
    search_input.send_keys(brand_keyword + " " + menu_keyword)
    search_input.send_keys(Keys.ENTER)

    # 결과 로딩 대기
    wait.until(EC.presence_of_element_located((By.CLASS_NAME, "btn_cate")))
    wait.until(EC.invisibility_of_element_located((By.CLASS_NAME, "layer_autocomplete")))

    # 가격 필터 열기 버튼 클릭 (JS 강제 클릭) - 그냥 click시 오류 발생
    price_filter_btn = wait.until(EC.element_to_be_clickable((By.CLASS_NAME, "btn_cate")))
    time.sleep(0.5)
    driver.execute_script("arguments[0].click();", price_filter_btn)

    # 최대 가격 입력
    max_price_input = wait.until(EC.presence_of_element_located((By.ID, "maxPrice")))
    max_price_input.clear()
    max_price_input.send_keys("7000")

    # 적용 버튼 클릭
    apply_btn = wait.until(EC.element_to_be_clickable((By.CLASS_NAME, "btn_apply")))
    driver.execute_script("arguments[0].click();", apply_btn)

    # 결과 로딩 대기
    wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, "link_info")))

    total = len(driver.find_elements(By.CLASS_NAME, "link_info"))

    for i in range(total):
        try:
            product_links = driver.find_elements(By.CLASS_NAME, "link_info")
            link = product_links[i]

            # 브랜드명 확인
            brand_elem = link.find_elements(By.CLASS_NAME, "txt_brand")
            if not brand_elem:
                continue
            brand = brand_elem[0].text

            if brand_keyword not in brand:
                continue

            # 메뉴명 확인
            title_elem = link.find_elements(By.CLASS_NAME, "txt_prdname")
            if not title_elem:
                continue
            title = title_elem[0].text

            title_clean = title.replace(" ", "")
            menu_keyword_clean = menu_keyword.replace(" ", "")

            # 제외 조건
            if (
                "+" in title or 
                len(title) > 20 or 
                menu_keyword_clean not in title_clean or
                "디카페인" in title or 
                "2잔" in title
            ):
                continue

            # 가격 확인
            price_elem = link.find_elements(By.CLASS_NAME, "num_price")
            price = price_elem[0].text if price_elem else "가격 없음"

            # 리스트에 저장
            brand_list.append(brand)
            menu_list.append(title)
            price_list.append(price)

        except Exception as e:
            print(f"오류 발생: {e}")
            continue

# 브랜드 x 메뉴 키워드 조합 반복 실행
for brand in brands:
    for menu in menu_keywords:
        try:
            search_and_extract(brand, menu)
        except Exception as e:
            print(f"검색 중 오류 발생: {e}")
        time.sleep(2)

# 종료
driver.quit()

# 데이터프레임 생성
df = pd.DataFrame({
    "브랜드": brand_list,
    "메뉴명": menu_list,
    "가격": price_list
})



# # Database에 저장

# remote = mysql.connector.connect(
#     host = "database-1.c3wcyagu0dve.ap-northeast-2.rds.amazonaws.com",
#     port = "3306",
#     user = "root",
#     password = "Rrnrrn1!",
#     database = "amrbase"
# )

# cur = remote.cursor(buffered=True)

# # coffee_price TABLE 생성
# cur.execute("CREATE TABLE coffee_price(Brand varchar(16), menu varchar(64), price int)")

# # TABLE에 데이터 insert
# sql = """INSERT INTO coffee_price VALUES (%s, %s, %s)"""

# for i, row in df.iterrows():
#     cur.execute(sql, tuple(row))
#     remote.commit()

# remote.commit()
# remote.close()

